In [1]:
import pandas as pd
import psycopg2 

In [27]:
df = pd.read_csv("./sales_data_sample.csv", sep=",", encoding='Latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   ORDERLINENUMBER   2823 non-null   int64  
 4   SALES             2823 non-null   float64
 5   ORDERDATE         2823 non-null   object 
 6   STATUS            2823 non-null   object 
 7   QTR_ID            2823 non-null   int64  
 8   MONTH_ID          2823 non-null   int64  
 9   YEAR_ID           2823 non-null   int64  
 10  PRODUCTLINE       2823 non-null   object 
 11  MSRP              2823 non-null   int64  
 12  PRODUCTCODE       2823 non-null   object 
 13  CUSTOMERNAME      2823 non-null   object 
 14  PHONE             2823 non-null   object 
 15  ADDRESSLINE1      2823 non-null   object 
 16  ADDRESSLINE2      302 non-null    object 


In [28]:
# change the datatype of order dae column
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])

In [29]:
#  dropping some unuseful columns for futher analysis
df = df.drop(['ADDRESSLINE1','ADDRESSLINE2','MONTH_ID','YEAR_ID',
              'PHONE','POSTALCODE','CONTACTFIRSTNAME','CONTACTLASTNAME','ORDERLINENUMBER'], axis=1)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ORDERNUMBER      2823 non-null   int64         
 1   QUANTITYORDERED  2823 non-null   int64         
 2   PRICEEACH        2823 non-null   float64       
 3   SALES            2823 non-null   float64       
 4   ORDERDATE        2823 non-null   datetime64[ns]
 5   STATUS           2823 non-null   object        
 6   QTR_ID           2823 non-null   int64         
 7   PRODUCTLINE      2823 non-null   object        
 8   MSRP             2823 non-null   int64         
 9   PRODUCTCODE      2823 non-null   object        
 10  CUSTOMERNAME     2823 non-null   object        
 11  CITY             2823 non-null   object        
 12  STATE            1337 non-null   object        
 13  COUNTRY          2823 non-null   object        
 14  TERRITORY        1749 non-null   object 

In [32]:
df = df[['ORDERNUMBER','QUANTITYORDERED','PRICEEACH',
         'ORDERDATE','CUSTOMERNAME','CITY','COUNTRY']]

In [33]:
df

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERDATE,CUSTOMERNAME,CITY,COUNTRY
0,10107,30,95.70,2003-02-24,Land of Toys Inc.,NYC,USA
1,10121,34,81.35,2003-05-07,Reims Collectables,Reims,France
2,10134,41,94.74,2003-07-01,Lyon Souveniers,Paris,France
3,10145,45,83.26,2003-08-25,Toys4GrownUps.com,Pasadena,USA
4,10159,49,100.00,2003-10-10,Corporate Gift Ideas Co.,San Francisco,USA
...,...,...,...,...,...,...,...
2818,10350,20,100.00,2004-12-02,Euro Shopping Channel,Madrid,Spain
2819,10373,29,100.00,2005-01-31,"Oulu Toy Supplies, Inc.",Oulu,Finland
2820,10386,43,100.00,2005-03-01,Euro Shopping Channel,Madrid,Spain
2821,10397,34,62.24,2005-03-28,Alpha Cognac,Toulouse,France


In [36]:
from psycopg2 import sql

# Define PostgreSQL connection details
host = "localhost"
database = "first_database"
user = "postgres"
password = "admin"
port = 5432  

# Function to create the database if it does not exist
def create_database_if_not_exists(db_name, user, password, host, port):
    # Connect to the PostgreSQL server (default database "postgres")
    conn = psycopg2.connect(
        dbname='postgres',
        user=user,
        password=password,
        host=host,
        port=port
    )
    conn.autocommit = True  # Required to create databases
    cur = conn.cursor()
    # Create the database if it does not exist
    cur.execute(
        sql.SQL("SELECT * FROM pg_database WHERE datname = %s"), [db_name])
    exists = cur.fetchone()
    if not exists:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(
            sql.Identifier(db_name)))
        print(f"Database {db_name} created successfully.")
    else:
        print(f"Database {db_name} already exists.")
    cur.close()
    conn.close()


# Create the database if it does not exist
create_database_if_not_exists(database, user, password, host, port)

# Establish connection to PostgreSQL
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)
cur = conn.cursor()

# Create a table (if not exists)
create_table_query = '''
CREATE TABLE IF NOT EXISTS sales_data (
    order_id TEXT,
    quantity_ordered INTEGER,
    price_each FLOAT,
    order_date DATE,
    customer_name TEXT,
    city TEXT,
    country TEXT
);
'''
cur.execute(create_table_query)

for _, row in df.iterrows():
    insert_query = '''
    INSERT INTO sales_data (order_id, quantity_ordered, price_each, order_date, customer_name, city, country)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    '''
    cur.execute(insert_query, tuple(row))

# Commit changes and close the connection
conn.commit()
cur.close()
conn.close()

print("Data loaded successfully into PostgreSQL")

Database first_database already exists.
Data loaded successfully into PostgreSQL
